In [40]:
# Install and load environment variables from .env file
%pip install python-dotenv

import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

print("Environment variables loaded successfully")

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Environment variables loaded successfully


## setup

In [41]:
%pip install --upgrade --quiet google-cloud-aiplatform

Note: you may need to restart the kernel to use updated packages.


In [42]:
%pip install -q -U google-generativeai

Note: you may need to restart the kernel to use updated packages.


In [43]:
!python -m pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable


In [44]:
%pip install google-genai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [45]:
%pip install pymupdf
%pip install markdownify

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [46]:
import pathlib
import textwrap
import google.generativeai as genai
#from google.colab import userdata
from IPython.display import display
from IPython.display import Markdown
import pandas as pd

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:

# Load API key from environment variable
api_key = os.getenv("GOOGLE_GENAI_API_KEY")

if not api_key:
    raise ValueError("GOOGLE_GENAI_API_KEY not found in environment variables. Please check your .env file.")

#genai.configure(api_key=api_key)


'\n# Load API key from environment variable\napi_key = os.getenv("GOOGLE_GENAI_API_KEY")\n\nif not api_key:\n    raise ValueError("GOOGLE_GENAI_API_KEY not found in environment variables. Please check your .env file.")\n\n#genai.configure(api_key=api_key)\n\n'

In [61]:
import IPython.display
from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Part,
)

In [ ]:
# Use the environment variable for Project ID
import os
import vertexai

PROJECT_ID = os.getenv("GOOGLE_CLOUD_PROJECT")
if not PROJECT_ID:
    raise ValueError("GOOGLE_CLOUD_PROJECT not found in environment variables. Please check your .env file.")

LOCATION = os.getenv("GOOGLE_CLOUD_REGION", "asia-northeast1")

vertexai.init(project=PROJECT_ID, location=LOCATION)

'\n# Use the environment variable for Project ID\nimport os\nimport vertexai\n\nPROJECT_ID = os.getenv("GOOGLE_CLOUD_PROJECT")\nif not PROJECT_ID:\n    raise ValueError("GOOGLE_CLOUD_PROJECT not found in environment variables. Please check your .env file.")\n\nLOCATION = os.getenv("GOOGLE_CLOUD_REGION", "asia-northeast1")\n\nvertexai.init(project=PROJECT_ID, location=LOCATION)\n'

In [63]:
MODEL_ID = 'gemini-3-flash-preview'

In [64]:
def get_prompt():
    prompt = """
あなたはJICA ODA事後評価報告書の分析者です。
報告書全文から「失敗要因（root cause）」のみを抽出し、JSON Lines（1行=1 JSON）で出力してください。

# 重要：失敗要因（root cause）の定義
- root cause = 「何が原因で、どんな負の結果（遅延/未達/未完成/停止/未活用/稼働低下/持続性低下）が起きたか」を説明できる原因
- 単なる現象・結果（例：事業費が上回った、遅れた、未完成）は原因ではないので出力禁止
- “時間がかかった”のような中間要因は、可能なら一段掘り下げて原因化する（例：承認遅延、人員不足、調達不調 等）

# 出力ポリシー（焦点を絞る）
- 最大8件まで。重要度の高い順（影響が大きい・繰り返し言及・構造的な原因を優先）
- 1行1原因（複数原因を混ぜない）
- evidence は原文からできるだけ逐語で抜粋（20〜160字）
- evidence から「原因→結果」が読み取れない場合は confidence を0.6未満にする
- root cause が見当たらない場合は1行だけ出して fail_factor="NONE"

# 出力項目（JSON Lines）
キー順固定：
title, fail_factor, impact, phase, actor, controllability, is_overrun_related, evidence, evidence_section, confidence

- title: プロジェクト名
- fail_factor: 原因（20〜80字、原因のみ。現象禁止）
- impact: fail_factor により生じた負の結果（20〜80字、具体。例：ガイド未完成、活動停止、指標検証困難、研修計画未作成 等）
- phase: "PLANNING" | "IMPLEMENTATION" | "OPERATION"
- actor: "PARTNER" | "JICA" | "VENDOR" | "EXOGENOUS" | "MULTIPLE"
- controllability: "HIGH" | "MEDIUM" | "LOW"
- is_overrun_related: true/false（遅延・コスト超過に直結するならtrue）
- evidence: 原文抜粋（逐語）
- evidence_section: 章名（不明なら"UNKNOWN"）
- confidence: 0.0〜1.0

# 出力禁止例（これは原因ではない）
- 「事業費が計画を上回った」「工期が延伸した」「未完成だった」など結果のみ
- 「課題がある」「不十分だった」だけで、何が原因か不明なもの

# 出力例（JSON Lines）
{"title":"プロジェクトA","fail_factor":"翻訳外注の予算と校閲人員が不足","impact":"M&Eガイドラインのパシュトゥー語版が未完成","phase":"IMPLEMENTATION","actor":"PARTNER","controllability":"MEDIUM","is_overrun_related":true,"evidence":"リソース不足(翻訳を外注するための資金及び最終的な編集や校閲のための人員の不足)のために完成しなかった","evidence_section":"プロジェクト目標及び上位目標の達成度","confidence":0.90}
"""
    return prompt

In [65]:
# お試し実行
from google import genai
import httpx
from google.genai import types

client = genai.Client(api_key=api_key)
pdf_file_uri = "https://www2.jica.go.jp/ja/evaluation/pdf/2020_0900816_4_f.pdf"

doc_data = httpx.get(pdf_file_uri).content

prompt = get_prompt()
pdf_file = Part.from_uri(pdf_file_uri, mime_type="application/pdf")
contents=[
  types.Part.from_bytes(
    data=doc_data,
    mime_type='application/pdf',
  ), prompt
]

response = client.models.generate_content(
    model=MODEL_ID,
    contents=contents,
)
print(response.text)

{"title":"リプロダクティブヘルスプロジェクト（フェーズ1・2）","fail_factor":"RHDにおけるデータ分析のための人材不足","impact":"RHサービスの提供・管理に関するデータの分析が十分に行われなかった","phase":"IMPLEMENTATION","actor":"PARTNER","controllability":"MEDIUM","is_overrun_related":false,"evidence":"RH サービスの提供・管理に関する指標の収集・報告は改善したものの、RHD によるデータ分析は人材不足を理由に十分に行われなかった","evidence_section":"2 有効性・インパクト","confidence":0.9}
{"title":"リプロダクティブヘルスプロジェクト（フェーズ1・2）","fail_factor":"政策策定支援の活動に時間を要したことによる作業の遅延","impact":"国家ISTガイドが編集中に留まり、研修計画の作成やISTの開始に至らなかった","phase":"IMPLEMENTATION","actor":"MULTIPLE","controllability":"MEDIUM","is_overrun_related":true,"evidence":"政策策定支援の活動に時間がかかったことで作業が遅れ、全国 IST ガイドは編集中であり、年間研修計画は作成されていなかった。そのため、同ガイドに基づく IST は開始されず","evidence_section":"プロジェクト目標及び上位目標の達成度","confidence":0.9}
{"title":"リプロダクティブヘルスプロジェクト（フェーズ1・2）","fail_factor":"翻訳外注資金及び最終編集・校閲のための人員の不足","impact":"M&Eガイドラインのパシュトゥー語版が完成しなかった","phase":"IMPLEMENTATION","actor":"PARTNER","controllability":"MEDIUM","is_overrun_related":false,"evidence":"パシュトゥー語版の新 M&E ガイドラインは、リソース不足（翻訳を外注するための資

In [66]:
pdf_file_uri = "https://www2.jica.go.jp/ja/evaluation/pdf/2019_0603827_4_f.pdf"

doc_data = httpx.get(pdf_file_uri).content

pdf_file = Part.from_uri(pdf_file_uri, mime_type="application/pdf")
contents=[
  types.Part.from_bytes(
    data=doc_data,
    mime_type='application/pdf',
  ), prompt
]

response = client.models.generate_content(
    model=MODEL_ID,
    contents=contents,
)
print(response.text)

{"title": "「結核対策プロジェクト」および「結核対策プロジェクトフェーズ 2」", "fail_factor": "アフガニスタン国内の治安悪化による日本人専門家の退避と渡航制限の発生", "impact": "現場での直接指導が制限され、プロジェクト期間が当初計画より1年間延長された", "phase": "IMPLEMENTATION", "actor": "EXOGENOUS", "controllability": "LOW", "is_overrun_related": true, "evidence": "治安悪化による専門家の退避があったりと、外部の条件が極めて不安定な中、アフガニスタンの結核対策を支援してきた。...事業期間は、フェーズ 1 は計画どおりであったが、フェーズ 2 は計画を上回った。", "evidence_section": "３.３.１.３ 事業期間", "confidence": 0.9}
{"title": "「結核対策プロジェクト」および「結核対策プロジェクトフェーズ 2」", "fail_factor": "2011年に発生した製造中止等による世界的な抗結核薬の在庫不足", "impact": "欠品回避のための緊急調達が必要となり、日本側事業費が計画を大幅に上回る要因となった", "phase": "IMPLEMENTATION", "actor": "EXOGENOUS", "controllability": "LOW", "is_overrun_related": true, "evidence": "2011 年には通常使用する抗結核薬のひとつの製造中止による世界的な不足に対する在庫切れの回避、および WHO 地中海事務所の 6 カ月療法への手続きの遅れによる抗結核薬の在庫切れの回避のため、やむを得ず抗結核薬を購入した", "evidence_section": "３.３.１.２ 事業費", "confidence": 0.9}
{"title": "「結核対策プロジェクト」および「結核対策プロジェクトフェーズ 2」", "fail_factor": "無償資金協力で建設された病院における暖房設備の不足と運営予算の未配分", "impact": "事業費を石油ストーブや患者用食料の購入に充てる必要が生じ、

## 取得対象の設定

In [14]:
input_path = "../df_check_99.csv"

df_html = pd.read_csv(input_path)

# 評価が悪いものだけ抽出
df_html = df_html[df_html['total_eval'] <= 2]

df_html["URL"] = df_html["file"]
df_html

,Unnamed: 0,file,country,eval_year,evaluator_group,evaluator,project_cost_plan,project_cost_act,プロジェクト期間開始_計画時,プロジェクト期間終了_計画時,...,貿易_flg_fix,農業一般_flg_fix,通信・放送一般_flg_fix,都市計画・土地造成_flg_fix,金融_flg_fix,開発_flg_fix,cost_rate,duration_rate,total_eval,URL
19,19,https://www2.jica.go.jp/ja/evaluation/pdf/2010...,ザンビア,2010,ザンビア事務所,NaN,7.770000e+08,7.750000e+08,2004-06-01,NaN,...,False,False,False,False,False,False,0.997426,NaN,2.0,https://www2.jica.go.jp/ja/evaluation/pdf/2010...
28,28,https://www2.jica.go.jp/ja/evaluation/pdf/2010...,モザンビーク,2010,OPMAC株式会社,三島光恵,2.167000e+09,2.147500e+09,2005-05-01,2007-07-01,...,False,False,False,False,False,False,0.991001,1.000000,2.0,https://www2.jica.go.jp/ja/evaluation/pdf/2010...
37,37,https://www2.jica.go.jp/ja/evaluation/pdf/2010...,ボリビア,2010,ボリビア事務所,NaN,6.830000e+08,6.830000e+08,2005-07-01,2006-07-11,...,False,True,False,False,False,False,1.000000,1.384615,2.0,https://www2.jica.go.jp/ja/evaluation/pdf/2010...
38,38,https://www2.jica.go.jp/ja/evaluation/pdf/2010...,ニカラグア,2010,財団法人国際開発高等教育機構,朝戸恵子,1.203900e+09,1.202177e+09,2005-06-01,2007-01-31,...,False,False,False,False,False,False,0.998569,0.952381,1.0,https://www2.jica.go.jp/ja/evaluation/pdf/2010...
48,48,https://www2.jica.go.jp/ja/evaluation/pdf/2010...,パキスタン,2010,財団法人国際開発高等教育機構,藤田伸子,1.259756e+09,1.009600e+09,2005-08-01,2007-01-14,...,False,False,False,False,False,False,0.801425,1.058824,1.0,https://www2.jica.go.jp/ja/evaluation/pdf/2010...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2164,2164,https://www2.jica.go.jp/ja/evaluation/pdf/2023...,ケニア,2023,アイ・シー・ネット株式会社,早瀬史麻,4.598000e+09,4.961000e+09,2012-06-01,2015-01-14,...,False,False,False,False,False,False,1.078947,4.250000,2.0,https://www2.jica.go.jp/ja/evaluation/pdf/2023...
2177,2177,https://www2.jica.go.jp/ja/evaluation/pdf/2023...,インド,2023,インド事務所,NaN,3.970000e+08,5.790000e+08,2013-05-01,2016-04-12,...,False,False,False,False,False,False,1.458438,1.611111,2.0,https://www2.jica.go.jp/ja/evaluation/pdf/2023...
2203,2203,https://www2.jica.go.jp/ja/evaluation/pdf/2023...,スリランカ,2023,株式会社グローバル・グループ21ジャパン,澤井克紀,1.883900e+10,1.648300e+10,2013-03-01,2016-12-12,...,False,False,False,False,False,False,0.874940,1.739130,2.0,https://www2.jica.go.jp/ja/evaluation/pdf/2023...
2220,2220,https://www2.jica.go.jp/ja/evaluation/pdf/2024...,ナイジェリア,2024,ナイジェリア事務所,NaN,3.700000e+08,3.880000e+08,2015-03-01,2018-03-05,...,False,False,False,False,False,False,1.048649,1.083333,2.0,https://www2.jica.go.jp/ja/evaluation/pdf/2024...


In [15]:
#df_html = df_html.head(10)

In [16]:
len(df_html)

527

## API呼び出し


In [17]:
def jsonize(jsonlikewords):
    import re
    import json

    # JSON文字列に変換
    json_text = jsonlikewords.replace("json", "")
    json_text = json_text.replace("\\n", "")
    json_text = json_text.replace("```", "")
    json_text = json_text.replace("    ", "")
    json_text = json_text.replace(",\n}", "\n}")
    

    # JSONとして読み込み
    json_data = json.loads(json_text)

    return json_data

In [18]:
import time
from google.api_core import retry
from google.api_core import exceptions as api_core_exceptions
import google.generativeai as genai
import google.genai.errors as genai_errors # ★ genai のエラーをインポート
import httpx
import logging

# ロギングの設定
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# リトライすべき google.api_core または google.genai のサーバーエラー/リソース枯渇エラーか判定する関数
def is_retryable_genai_server_error(exc):
    """google.api_core または google.genai のリトライすべき例外か判定"""
    # google.api_core の例外をチェック (500, 503, 504, 429)
    if isinstance(exc, (api_core_exceptions.InternalServerError,
                       api_core_exceptions.ServiceUnavailable,
                       api_core_exceptions.DeadlineExceeded,
                       api_core_exceptions.ResourceExhausted)):
        logging.warning(f"Detected google.api_core retryable error ({type(exc).__name__}). Retrying...")
        return True

    # ★ google.genai.errors の例外をチェック
    # ServerError (500, 503 など) をリトライ対象に含める
    if isinstance(exc, genai_errors.ServerError):
        # ServerError は 5xx 系エラーを示すため、基本的にリトライ対象とする
        logging.warning(f"Detected google.genai.errors.ServerError. Retrying...")
        return True
    # ResourceExhaustedError (429) もリトライ対象に含める
    if isinstance(exc, genai_errors.ResourceExhaustedError):
         logging.warning(f"Detected google.genai.errors.ResourceExhaustedError (Quota Exceeded?). Retrying...")
         return True
    # 他にもリトライしたい genai のエラーがあればここに追加
    # 例: if isinstance(exc, genai_errors.DeadlineExceededError): return True

    return False

# リトライすべき httpx のエラーか判定する関数
def is_retryable_http_error(exc):
    """httpx の 5xx系 または 429 HTTPエラーかどうかを判定"""
    if isinstance(exc, httpx.HTTPStatusError):
        status_code = exc.response.status_code
        is_5xx_error = status_code >= 500 and status_code < 600
        is_429_error = status_code == 429 # Too Many Requests
        if is_5xx_error or is_429_error:
             logging.warning(f"Detected HTTP Error (Status: {status_code}). Retrying...")
             return True
    return False

# 総合的なリトライ判定関数
def should_retry(exc):
    """リトライすべき例外かどうかを総合的に判断"""
    # is_retryable_genai_server_error で True ならリトライ
    if is_retryable_genai_server_error(exc):
        return True
    # is_retryable_http_error で True ならリトライ
    if is_retryable_http_error(exc):
        return True
    # google.api_core の RetryError (内部リトライからのエラー)
    if isinstance(exc, api_core_exceptions.RetryError):
         logging.warning(f"Google API Core RetryError detected. Retrying...")
         return True

    # 上記以外はリトライしない
    # Non-retryable のログレベルを WARNING に変更（エラーではないため）
    logging.warning(f"Non-retryable error occurred: {type(exc).__name__} - {exc}")
    return False


In [19]:
@retry.Retry(
    predicate=should_retry,      # リトライ条件を指定する関数
    initial=1.0,               # 最初のリトライまでの遅延時間（秒）(google.api_core の推奨は initial)
    maximum=60.0,              # 最大遅延時間（秒）(google.api_core の推奨は maximum)
    multiplier=2.0,            # 遅延時間の倍率 (google.api_core の推奨は multiplier)
    deadline=300.0,            # 全体の試行の最大時間（秒） (google.api_core の推奨は deadline)
    # on_error コールバックでリトライ時に追加処理も可能
    # on_error=lambda exc: logging.warning(f"Retrying due to {exc}")
)
def generate_content_with_retry(url: str, model_id: str):
    """
    PDFデータとプロンプトを使用してGemini APIを呼び出し、コンテンツを生成する（リトライ付き）。

    Returns:
        Gemini APIからのレスポンスオブジェクト。

    Raises:
        google.api_core.exceptions.RetryError: 指定されたリトライ回数を超えても成功しなかった場合。
        Exception: その他の予期しないエラー。
    """
    try:
        logging.info(f"Attempting to generate content with model: {model_id}")

        doc_data = httpx.get(url).content
        pdf_file = Part.from_uri(pdf_file_uri, mime_type="application/pdf")
        contents=[
          types.Part.from_bytes(
            data=doc_data,
            mime_type='application/pdf',
          ), prompt
        ]

        response = client.models.generate_content(
            model=model_id,
            contents=contents,
        )

        logging.info("Successfully generated content.")
        return response

    except (api_core_exceptions.GoogleAPIError, httpx.HTTPStatusError) as e:
        # should_retry で判定されるので、ここではログ出力のみ行い再raiseする
        # (should_retryでFalseが返れば、このraiseが最終的なエラー送出となる)
        logging.error(f"Error during API call or data processing: {e}")
        raise e
    except Exception as e:
        # 予期しないその他のエラー
        logging.exception(f"An unexpected error occurred in generate_content_with_retry: {e}")
        # 予期しないエラーはリトライさせずに上位に投げる
        raise

In [20]:
import json
import pandas as pd


def create_df_from_jsonlikewords(jsonlikewords):
    # json_out: JSON Lines（1行=1JSON）の文字列が入っている前提
    lines = [ln.strip() for ln in jsonlikewords.splitlines() if ln.strip()]

    records = []
    for i, ln in enumerate(lines, start=1):
        try:
            records.append(json.loads(ln))
        except json.JSONDecodeError as e:
            raise ValueError(f"Invalid JSON at line {i}: {e}\nLINE={ln[:200]}...") from e

    df = pd.DataFrame.from_records(records)

    # 任意：列順を固定したい場合（存在する列だけ）
    cols = [
        "title","fail_factor","impact","phase","actor","controllability",
        "is_overrun_related","evidence","evidence_section","confidence"
    ]
    df = df[[c for c in cols if c in df.columns]]

    return df

In [21]:
import pandas as pd
import json
import grpc
import time
from google.api_core.exceptions import InvalidArgument

prompt = get_prompt()
error=[]
path = "df_fail_factor"
df_out = pd.read_csv(f'out_csv/{path}.csv')
#df_out = pd.DataFrame()
j = 0

for i,row in df_html.iterrows():
    j += 1
    print(f"processing {j} out of {len(df_html)}")
    # httpの場合ダウンロードできないため置換
    f = row["URL"]
    f = f.replace("http:", "https:")

    retry_flg = len(df_out) > 0
    skip_flg = False

    if retry_flg:
      skip_flg = f in df_out["file"].values

    if skip_flg:
        print(f"already processed {f}")
    else:
        try:
          pdf_file_name = f.split('/')[-1]
          print(f"processing {f}")

          response = generate_content_with_retry(url=f,model_id=MODEL_ID)

          df = create_df_from_jsonlikewords(response.text)
          df["file"] = f
          df_out=pd.concat([df_out, df])
          if j % 50 == 0:
              df_out.to_csv(f"../out_csv/{path}.csv", index=False)
              print(f"save df succeessed {pdf_file_name}")

        except api_core_exceptions.RetryError as e:
          logging.error(f"Failed to generate content after multiple retries: {e}")
        except Exception as e:
          logging.error(f"An error occurred during the process: {e}", exc_info=True) # トレースバックも出力

        except grpc.RpcError as e:
          error.append(f)
          print(f"failed RpcError {f}")

        except InvalidArgument:
          print(f"too many token {f}")

        except(ValueError):
          error.append(f)
          print(f"failed ValueError {f}")

2025-12-31 00:34:22,126 - INFO - Attempting to generate content with model: gemini-3-flash-preview
2025-12-31 00:34:22,207 - INFO - HTTP Request: GET https://www2.jica.go.jp/ja/evaluation/pdf/2011_PE-P27_4_f.pdf "HTTP/1.1 200 OK"


processing 1 out of 527
already processed https://www2.jica.go.jp/ja/evaluation/pdf/2010_0407000_4_f.pdf
processing 2 out of 527
already processed https://www2.jica.go.jp/ja/evaluation/pdf/2010_0501300_4_f.pdf
processing 3 out of 527
already processed https://www2.jica.go.jp/ja/evaluation/pdf/2010_0507800_4_f.pdf
processing 4 out of 527
already processed https://www2.jica.go.jp/ja/evaluation/pdf/2010_0508600_4_f.pdf
processing 5 out of 527
already processed https://www2.jica.go.jp/ja/evaluation/pdf/2010_0511500_4_f.pdf
processing 6 out of 527
already processed https://www2.jica.go.jp/ja/evaluation/pdf/2010_0601477_4_f.pdf
processing 7 out of 527
already processed https://www2.jica.go.jp/ja/evaluation/pdf/2010_0601488_4_f.pdf
processing 8 out of 527
already processed https://www2.jica.go.jp/ja/evaluation/pdf/2010_0601701_4_f.pdf
processing 9 out of 527
already processed https://www2.jica.go.jp/ja/evaluation/pdf/2010_0601706_4_f.pdf
processing 10 out of 527
already processed https://www2

2025-12-31 00:34:22,367 - INFO - AFC is enabled with max remote calls: 10.
2025-12-31 00:34:46,303 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-3-flash-preview:generateContent "HTTP/1.1 200 OK"
2025-12-31 00:34:46,311 - INFO - Successfully generated content.
2025-12-31 00:34:46,312 - WARNING - Warning: there are non-text parts in the response: ['thought_signature'], returning concatenated text result from text parts. Check the full candidates.content.parts accessor to get the full model response.
2025-12-31 00:34:46,333 - INFO - Attempting to generate content with model: gemini-3-flash-preview
2025-12-31 00:34:46,419 - INFO - HTTP Request: GET https://www2.jica.go.jp/ja/evaluation/pdf/2015_PE-P29_4_f.pdf "HTTP/1.1 200 OK"


processing 59 out of 527
already processed https://www2.jica.go.jp/ja/evaluation/pdf/2011_PG-P14_4_f.pdf
processing 60 out of 527
already processed https://www2.jica.go.jp/ja/evaluation/pdf/2011_PH-P123_4_f.pdf
processing 61 out of 527
already processed https://www2.jica.go.jp/ja/evaluation/pdf/2011_PH-P187_4_f.pdf
processing 62 out of 527
already processed https://www2.jica.go.jp/ja/evaluation/pdf/2011_PH-P196_4_f.pdf
processing 63 out of 527
already processed https://www2.jica.go.jp/ja/evaluation/pdf/2011_PH-P227_4_f.pdf
processing 64 out of 527
already processed https://www2.jica.go.jp/ja/evaluation/pdf/2012_0402500_4_f.pdf
processing 65 out of 527
already processed https://www2.jica.go.jp/ja/evaluation/pdf/2012_0407700_4_f.pdf
processing 66 out of 527
already processed https://www2.jica.go.jp/ja/evaluation/pdf/2012_0503600_4_f.pdf
processing 67 out of 527
already processed https://www2.jica.go.jp/ja/evaluation/pdf/2012_0508500_4_f.pdf
processing 68 out of 527
already processed http

2025-12-31 00:34:46,538 - INFO - AFC is enabled with max remote calls: 10.
2025-12-31 00:35:11,690 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-3-flash-preview:generateContent "HTTP/1.1 200 OK"
2025-12-31 00:35:11,692 - INFO - Successfully generated content.
2025-12-31 00:35:11,693 - WARNING - Warning: there are non-text parts in the response: ['thought_signature'], returning concatenated text result from text parts. Check the full candidates.content.parts accessor to get the full model response.


processing 242 out of 527
already processed https://www2.jica.go.jp/ja/evaluation/pdf/2015_PE-P37_4_f.pdf
processing 243 out of 527
already processed https://www2.jica.go.jp/ja/evaluation/pdf/2015_SL-P82_4_f.pdf
processing 244 out of 527
already processed https://www2.jica.go.jp/ja/evaluation/pdf/2016_0506600_4_f.pdf
processing 245 out of 527
already processed https://www2.jica.go.jp/ja/evaluation/pdf/2016_0603238_4_f.pdf
processing 246 out of 527
already processed https://www2.jica.go.jp/ja/evaluation/pdf/2016_0700536_4_f.pdf
processing 247 out of 527
already processed https://www2.jica.go.jp/ja/evaluation/pdf/2016_0700809_4_f.pdf
processing 248 out of 527
already processed https://www2.jica.go.jp/ja/evaluation/pdf/2016_0700927_4_f.pdf
processing 249 out of 527
already processed https://www2.jica.go.jp/ja/evaluation/pdf/2016_0701031_4_f.pdf
processing 250 out of 527
already processed https://www2.jica.go.jp/ja/evaluation/pdf/2016_0701107_4_f.pdf
processing 251 out of 527
already proce

In [22]:
df_out

,Unnamed: 0.1,Unnamed: 0,title,fail_factor,impact,phase,actor,controllability,is_overrun_related,evidence,evidence_section,confidence,file
0,0.0,0.0,北部州地下水開発計画,鉄鋼価格およびザンビア・クワチャの対米ドルレートの上昇,掘削された深井戸の数量が、計画されていた175本に対し実績は163本にとどまった,IMPLEMENTATION,EXOGENOUS,LOW,False,アウトプットである深井戸の数量は、鉄鋼価格およびザンビア・クワチャの対米ドルレートの上昇のた...,2 効率性,1.0,https://www2.jica.go.jp/ja/evaluation/pdf/2010...
1,1.0,1.0,北部州地下水開発計画,住民の関与不足による、維持管理費（徴収率12～52％）の徴収不足,施設の運営・維持管理にかかる財務面に問題が生じ、事業の持続性に影響を及ぼしている,OPERATION,PARTNER,MEDIUM,False,住民からの維持管理費の徴収率が比較的低いため、財務の一部に問題が見られたほか、故障したハンド...,Ⅱ 評価結果,0.9,https://www2.jica.go.jp/ja/evaluation/pdf/2010...
2,2.0,2.0,北部州地下水開発計画,一部の村落レベル維持管理組織（V-WASHE）の不活性化,故障したハンドポンプの放置や、維持管理状況に関する最新情報の入手困難を招いている,OPERATION,PARTNER,MEDIUM,False,2010 年 1 月時点で故障したハンドポンプや活動していない村落レベル維持管理組織があり、...,4 持続性,0.9,https://www2.jica.go.jp/ja/evaluation/pdf/2010...
3,3.0,3.0,北部州地下水開発計画,給水状況に関するモニタリング・情報管理システムの未整備,事後評価時点において安全な水の供給量データが入手できず、事業効果の検証が困難となった,OPERATION,PARTNER,HIGH,False,事後評価時点の安全な水の供給量はデータが入手できず確認できなかったため、効果発現に一部問題が...,Ⅱ 評価結果,0.8,https://www2.jica.go.jp/ja/evaluation/pdf/2010...
4,4.0,4.0,北部州地下水開発計画,地下水における高い鉄分含有量の発生（自然要因および掘削孔の浸食等）,住民の井戸水利用に悪影響を及ぼし、事業によって得られる効果の最大化を阻害している,OPERATION,EXOGENOUS,LOW,False,北部州の地下水は鉄含有量が多いため、住民の井戸水利用に影響している。...事業効果の最大化を...,3 有効性･インパクト,0.9,https://www2.jica.go.jp/ja/evaluation/pdf/2010...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,NaN,NaN,地方都市上下水道整備事業（II）,現地の厳しい気象条件（落雷等）や運営維持管理能力を十分に考慮しない不適切な機材選定,遠隔監理システムが頻繁に故障し修理不能となり、また凝集池パネルが材質不適切で損傷・取り外され...,PLANNING,VENDOR,HIGH,False,遠隔監理システムは計器類を始めとした機器の各部が落雷等により頻繁に故障し、修理ができないため...,（１） イキトス,1.0,https://www2.jica.go.jp/ja/evaluation/pdf/2015...
4,NaN,NaN,地方都市上下水道整備事業（II）,民営化や料金値上げを懸念した住民の反対運動と、住民の感情を刺激した前処理施設の位置変更,シクアニでの円借款利用が断念され国家予算への切り替えが必要となり、また工事の中断や防臭設備の...,IMPLEMENTATION,EXOGENOUS,LOW,True,円借款の転貸を受けて上下水道整備をすることが事業の民営化や水道料金の大幅な値上げにつながるこ...,（３） シクアニ,0.9,https://www2.jica.go.jp/ja/evaluation/pdf/2015...
5,NaN,NaN,地方都市上下水道整備事業（II）,水道料金の低据え置き、高い無収水率（盗水等）、多額の人件費による脆弱な財務構造,イキトスやシクアニの上下水道公社が営業赤字に陥り、円借款の返済が滞るなど、事業の持続性に深刻...,OPERATION,PARTNER,MEDIUM,False,イキトス上下水道公社では、水道料金の低さ、多くの盗水を伴う無収水率の高さ、電力値上等による水...,３.５.３ 運営・維持管理の財務,1.0,https://www2.jica.go.jp/ja/evaluation/pdf/2015...
6,NaN,NaN,地方都市上下水道整備事業（II）,公社の給与水準が低く、高度な処理施設を管理できる専門技術者を雇用・維持できないこと,シクアニの下水処理場の適切な運営・維持管理体制が構築できず、技術的な持続性に懸念が生じている,OPERATION,PARTNER,LOW,False,下水処理場の運営・維持管理には適切な能力を備えた技師が必要であるが、給与水準が低い同公社では...,３.５.２ 運営・維持管理の技術,0.9,https://www2.jica.go.jp/ja/evaluation/pdf/2015...


In [23]:
df_out.to_csv(f"out_csv/{path}.csv")

In [24]:
len(df_out['file'].unique())

527